In [ ]:
# Downloading the Data

import os
import tarfile
import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()

# Loading the Data

import pandas as pd
 
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()

In [ ]:
# Data exploration

housing.info()

#housing["ocean_proximity"].value_counts()

housing.describe()

%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
# Writing a function to create the test set

import numpy as np

def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

# Implementing hash computation to split data in train, test set
from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

housing_with_id = housing.reset_index() # need to add index column since dataset doesn't have one
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

# Using latitude and longitude features to build a stable id
housing_with_id["id"] = housing["longitude"] *1000 + housing["latitude"]
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "id")

# Using sklearn's train_test_split to create test set
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
# Ensure stratified sampling to avoid sampling bias casuing the creation of a non-representative train/test set
# Median income is an important attribute to predict the median home prices
# Create income categories

housing["income_cat"] = pd.cut(housing["median_income"], 
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

housing["income_cat"].hist()

# Create stratified train/test sets
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

# Checking income category proportions to see if split worked
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

# Removing the income_cat attribute to return data back to original state
for set_ in (strat_train_set, strat_test_set):
    set_.drop ("income_cat", axis=1, inplace=True)

In [ ]:
# Visualizations

housing = strat_train_set.copy() # Creating a copy of the trianing set to explore without disturbing the actual training set

housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1) # High-density areas stand out: Bay Area, Los Angeles, San Diego, Central Valley: Sacramento and Fresno

housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, # Regions along the coast with high population density generally have more expensive homes
    s=housing["population"]/100, label="population", figsize = (10,7),
    c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True)
plt.legend()

from pandas.plotting import scatter_matrix # To plot promising numerical attributes with other numerical attributes to look for correlations
attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8)) # The most promising attribute to predict median house value is the median income

housing.plot(kind="scatter", x="median_income", y="median_house_value", # Zooming in on the median_income vs median_home_value plot reveals a strong correlation, and a clear upward trend
             alpha=0.1)                                                 # Some horizontal lines at different price levels may need to be dealt with to avoid algorithm learning to reproduce them

In [ ]:
# Creating some more informative attributes

housing["rooms_per_household"] = housing["total_rooms"]/housing["households"] # Getting the rooms per home as opposed to the whole district
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"] # Getting the ratio of number bedrooms to all rooms
housing["population_per_household"] = housing["population"]/housing["households"] # Getting the average number of people living in one home

# Checking correlation matrix again to see if the new attributes were informative
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False) # The bedrooms_per_room attribute seems to be much more correlated to the median house value than the total number to bedrooms and                                                                  rooms per district, and appears to be negatively correlated (that is as the number of bedrooms relative to the number of total                                                                    rooms decreases, the higher the median home value). The rooms_per_household attribute also seems to have a reasonable                                                                             correlation to the median home value (that is the larger the house, the more expensive it tends to be)

In [ ]:
# Data Cleaning and general data preparation for applying the algorithms

housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

from sklearn.impute import SimpleImputer # Creating an imputer instance to fill in missing values in total_bedrooms attribute

imputer = SimpleImputer(strategy="median")
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)
imputer.statistics_

X = imputer.transform(housing_num) # Fitting imputer to fill in missing values in dataset
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index) # Transforming transformed dataset back into a DataFrame

###########

housing_cat = housing[["ocean_proximity"]] # Only one categorical attribute
housing_cat.head(5)

from sklearn.preprocessing import OrdinalEncoder # Converting the tex categories to numerical categories

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10] # One major flaw of this method is when some ML algorithms use distance measures to assign similarity to certain values so categories 0 and 1 will be
                         # considered more similar than 0 and 4, when the ordinality of a category does not matter. Another method to use would be one-hot encoding.

from sklearn.preprocessing import OneHotEncoder # Creating instance of one-hot encoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot # Sparse matrix due to the large amount of 0s

In [ ]:
# Custom Transformer for adding attributes

from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAtrributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, bedrooms_ix] / X[:, population_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAtrributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [ ]:
# Pipeline for transformations for all attributes (instead of completing steps one by one)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('attribs_adder', CombinedAtrributesAdder()),
        ('std_scaler', StandardScaler())
    ])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs)
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [27]:
# Training the Models
 
from sklearn.linear_model import LinearRegression # Trying vanilla linear regression just to get a basline estimate of a model's performance
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse # Model performs quite poorly, typical price of a housing is predicted off by $68,358; model is underfitting

from sklearn.tree import DecisionTreeRegressor # Trying decision trees

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse # Sign of model badly overfitting the data, need to use cross-validation to measure performance better

from sklearn.model_selection import cross_val_score # Trying vanilla 10-fold cv

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation:", scores.std())
display_scores(tree_rmse_scores) # Decision tree not performing that well either, training score is much lower than the validation score, hence still overfitting 

lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores) # Checking to see Linear Regression performance usign cv, actually performs better than the tree (tree overfitting badly)

from sklearn.ensemble import RandomForestRegressor # Trying an ensemble method to see if any improvement in model performance (RF is a relatively good performer)

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores) # The RF performed much better but the training score is still much lower than the score on the validation sets, model still overfitting

Scores: [69952.92350963 66032.28238426 74735.96803598 74339.50445108
 72521.04875707 70577.24128527 72103.27044767 67322.5855089
 73882.28500031 70494.66996301]
Mean: 71196.17793431657
Standard Deviation: 2762.1603746626943
Scores: [66271.03705179 66631.22287781 75107.59770393 73108.09877086
 67923.30860402 70805.48969234 65075.7320055  67785.94213353
 71399.47035119 67097.38137351]
Mean: 69120.5280564475
Standard Deviation: 3126.700940589302
Scores: [50033.90080886 47346.89540929 49756.93776837 53272.76693108
 50345.15709479 53073.97900908 48801.08876381 48885.03136826
 53624.89660643 50501.2677463 ]
Mean: 50564.192150625815
Standard Deviation: 2006.669787745942


In [28]:
# Model Tuning

from sklearn.model_selection import GridSearchCV 

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [3, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
  ]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)
grid_search.best_params_
grid_search.best_estimator_

cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
# The best model was obtained by setting the max_features hyperparameter to 8 and the n_estimators parameter to 30, which gave a RMSE score of 50295, slightly better than the default hyperparameters we used for the earlier model. Perhaps try RandomizedSearchCV instead to get an even larger field of hyperparameters, possibly obtaining better hyperparameter values

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [3, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [44]:
# Model Inspection

feature_importances = grid_search.best_estimator_.feature_importances_
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)
# Interestingly only one ocean-proximity category (INLAND) is useful. Median Income is, predictably, the most useful feature

[(0.3932638024489401, 'median_income'),
 (0.14571475857006594, 'INLAND'),
 (0.10775214882964577, 'pop_per_hhold'),
 (0.06899322759117771, 'bedrooms_per_room'),
 (0.0677244900810488, 'longitude'),
 (0.0640050814711206, 'latitude'),
 (0.043621086346769804, 'housing_median_age'),
 (0.035139033305924675, 'rooms_per_hhold'),
 (0.015876034947622082, 'population'),
 (0.015525788236839264, 'total_rooms'),
 (0.01529608745636973, 'households'),
 (0.014426550659181128, 'total_bedrooms'),
 (0.007834015539202788, '<1H OCEAN'),
 (0.002724381711702957, 'NEAR OCEAN'),
 (0.0020949247586501112, 'NEAR BAY'),
 (8.588045738466513e-06, 'ISLAND')]

In [54]:
# Final Model Evaluation

final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test) # Being careful not to fit pipeline on test set

final_predicitons = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predicitons)
final_rmse = np.sqrt(final_mse)
final_rmse # Reasonable result based on validation score

48043.695875746096